In [9]:
import pandas as pd
from pandas.io import sql
import sqlite3
import timeit

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [10]:
pwd

'/Users/nicholasbeaudoin/Desktop/ShotLink Analytics'

In [25]:
conn = sqlite3.connect('Shotlink.db')

In [26]:
# col_names = ['Tour Code', 'TourDescription', 'Year', 'Tourn.#', 'Player#',
#        'Course#', 'PermanentTournament#', 'PlayerFirstName',
#        'PlayerLastName', 'Round', 'TournamentName', 'CourseName',
#        'Hole', 'HoleScore', 'ParValue', 'Yardage', 'Shot',
#        'ShotType(S/P/D)', '#ofStrokes', 'FromLocation(Scorer)',
#        'FromLocation(Enhanced)', 'To Location(Scorer)',
#        'ToLocation(Enhanced)', 'Distance', 'DistancetoPin',
#        'IntheHoleFlag', 'AroundtheGreenFlag', '1stPuttFlag',
#        'DistancetoHoleaftertheShot', 'Time', 'Lie', 'Elevation',
#        'Slope', 'X Coordinate', 'Y Coordinate', 'Z Coordinate',
#        'DistancefromCenter', 'DistancefromEdge', 'Date', 'Left/Right',
#        'StrokesGained/Baseline', 'StrokesGainedCategory',
#        'RecoveryShot']

In [27]:
# years = list(range(2003, 2019))

# for i in years:

#     print('Working on year {} data'.format(i))
    
#     # Bring in data from local machine into pandas
#     df = pd.read_csv("data/tour_{}.TXT".format(i), sep=';', names = col_names, low_memory=False, encoding='latin-1')

#     print("Data loaded in dataframe")
    
#     # Drop first row because we created new headers
#     df.drop(0, inplace=True)

#     # Move to SQL DB
#     df.to_sql("Shotlink".format(i), con=conn, if_exists='append', index=False)
    
#     # Status check
#     print('Uploaded to DB')
    
#     # Delete DF from local memory
#     del df

Working on year 2003 data
Data loaded in dataframe
Uploaded to DB
Working on year 2004 data
Data loaded in dataframe
Uploaded to DB
Working on year 2005 data
Data loaded in dataframe
Uploaded to DB
Working on year 2006 data
Data loaded in dataframe
Uploaded to DB
Working on year 2007 data
Data loaded in dataframe
Uploaded to DB
Working on year 2008 data
Data loaded in dataframe
Uploaded to DB
Working on year 2009 data
Data loaded in dataframe
Uploaded to DB
Working on year 2010 data
Data loaded in dataframe
Uploaded to DB
Working on year 2011 data
Data loaded in dataframe
Uploaded to DB
Working on year 2012 data
Data loaded in dataframe
Uploaded to DB
Working on year 2013 data
Data loaded in dataframe
Uploaded to DB
Working on year 2014 data
Data loaded in dataframe
Uploaded to DB
Working on year 2015 data
Data loaded in dataframe
Uploaded to DB
Working on year 2016 data
Data loaded in dataframe
Uploaded to DB
Working on year 2017 data
Data loaded in dataframe
Uploaded to DB
Working on

## Generic one-player query

In [38]:
df = sql.read_sql("""SELECT * FROM Shotlink 
                WHERE PlayerLastName = 'Woods'""", con=conn)
df.head()
print(len(df))

39140


## Data Cleaning

In [30]:
# Create full player names
df['PlayerFullName'] = df['PlayerLastName'] + ", " + df['PlayerFirstName']

# Convert to numeric values
df['Round'] = pd.to_numeric(df['Round'], errors='coerce')
df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')
df['Hole'] = pd.to_numeric(df['Hole'], errors='coerce')
df['HoleScore'] = pd.to_numeric(df['HoleScore'], errors='coerce')
df['Yardage'] = pd.to_numeric(df['Yardage'], errors='coerce')
df['Shot'] = pd.to_numeric(df['Shot'], errors='coerce')
df['#ofStrokes'] = pd.to_numeric(df['#ofStrokes'], errors='coerce')
df['DistancetoPin'] = pd.to_numeric(df['DistancetoPin'], errors='coerce')
df['DistancetoHoleaftertheShot'] = pd.to_numeric(df['DistancetoHoleaftertheShot'], errors='coerce')
df['DistancefromEdge'] = pd.to_numeric(df['DistancefromEdge'], errors='coerce')
df['StrokesGained/Baseline'] = pd.to_numeric(df['StrokesGained/Baseline'], errors='coerce')

# Change NaN to zeros
df.fillna(0, inplace=True)

In [31]:
df.isnull().sum()

Tour Code                     0
TourDescription               0
Year                          0
Tourn.#                       0
Player#                       0
Course#                       0
PermanentTournament#          0
PlayerFirstName               0
PlayerLastName                0
Round                         0
TournamentName                0
CourseName                    0
Hole                          0
HoleScore                     0
ParValue                      0
Yardage                       0
Shot                          0
ShotType(S/P/D)               0
#ofStrokes                    0
FromLocation(Scorer)          0
FromLocation(Enhanced)        0
To Location(Scorer)           0
ToLocation(Enhanced)          0
Distance                      0
DistancetoPin                 0
IntheHoleFlag                 0
AroundtheGreenFlag            0
1stPuttFlag                   0
DistancetoHoleaftertheShot    0
Time                          0
Lie                           0
Elevatio

In [32]:
df.dtypes

Tour Code                      object
TourDescription                object
Year                           object
Tourn.#                        object
Player#                        object
Course#                        object
PermanentTournament#           object
PlayerFirstName                object
PlayerLastName                 object
Round                           int64
TournamentName                 object
CourseName                     object
Hole                            int64
HoleScore                       int64
ParValue                       object
Yardage                         int64
Shot                            int64
ShotType(S/P/D)                object
#ofStrokes                      int64
FromLocation(Scorer)           object
FromLocation(Enhanced)         object
To Location(Scorer)            object
ToLocation(Enhanced)           object
Distance                        int64
DistancetoPin                   int64
IntheHoleFlag                  object
AroundtheGre

In [33]:
# Normalize Distances

df['DistanceYds'] = df['Distance']//36
df['DistancetoPinYds'] = df['DistancetoPin']//36
df['DistancetoHoleaftertheShotYds'] = df['DistancetoHoleaftertheShot']//36
#df['DistancefromCenterYds'] = df['DistancefromCenter']//36
df['DistancefromEdgeYds'] = df['DistancefromEdge']//36

df['DistanceFt'] = round(df['Distance']/12, 1)
df['DistancetoPinFt'] = round(df['DistancetoPin']/12, 1)
df['DistancetoHoleaftertheShotFt'] = round(df['DistancetoHoleaftertheShot']/12, 1)
#df['DistancefromCenterFt'] = round(df['DistancefromCenter']/12, 1)
df['DistancefromEdgeFt'] = round(df['DistancefromEdge']/12, 1)